# Ejemplo 1 - clustering de colores

Vamos a probar el algoritmo [k-means de Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) para agrupar los colores de una imagen.

<br>

En primer lugar cargamos y mostramos la imagen a procesar:

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image


# Descargamos una imagen de prueba

!wget -q http://www.dlsi.ua.es/~jgallego/deepraltamira/sample_lenna.jpg



# Abrimos la imagen y la mostramos

imagen = Image.open("sample_lenna.jpg")


plt.imshow( imagen )
plt.axis("off")
plt.show()


<br>

A continuación escribimos el código para procesar una imagen mediante el método k-means:

<br>

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# ---------------------------------------------------------------------
def kmeans_image(imagen, num_colors):
    # Convertir la imagen a una matriz numpy
    image_np = np.array(imagen)

    # Obtener las dimensiones de la imagen
    height, width, _ = image_np.shape

    # Reshape la matriz para que tenga la forma (n_pixels, n_features)
    image_np_flat = image_np.reshape(-1, 3)

    # Aplicar el algoritmo K-means
    kmeans = KMeans(n_clusters=num_colors, init='k-means++', n_init='auto', random_state=42)
    kmeans.fit(image_np_flat)

    # Obtener los centroides y las etiquetas de los clusters
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    # Crear una imagen vacía para almacenar los colores agrupados
    clustered_image = np.zeros_like(image_np_flat)

    # Asignar cada píxel al centroide más cercano y asignarle su color
    for i in range(len(clustered_image)):
        clustered_image[i] = centroids[labels[i]]

    # Restaurar la forma original de la imagen
    clustered_image = clustered_image.reshape(height, width, 3)

    return clustered_image


print("\n¡Función 'kmeans_image' definida!\n")


<br>

Por último utilizamos la función anterior para procesar la imagen que habíamos cargado al inicio:

<br>

In [ ]:
# ---------------------------------------------------------------------
# Ejemplo de uso

num_colors = 4

clustered = kmeans_image( imagen, num_colors )


# Mostrar la imagen original y la imagen agrupada
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow( imagen )
ax[0].set_title("Imagen original")
ax[0].axis("off")

ax[1].imshow( clustered.astype(np.uint8) )
ax[1].set_title("Imagen agrupada con K-means")
ax[1].axis("off")

plt.show()


# Ejemplo 2 - Clustering MNIST

Vamos a aplicar clustering para agrupar los dígitos de la base de datos [MNIST](https://en.wikipedia.org/wiki/MNIST_database) según su apariencia.

<br>

Empezamos por cargar los datos y mostrar un dígito de ejemplo:

In [ ]:
import pandas as pd
import numpy as np

# Importamos la base de datos MNIST que viene con Colab de ejemplo.
# Para esto usamos Pandas:

data = pd.read_csv('/content/sample_data/mnist_test.csv', header=None)


# Guardamos los atributos de los datos en X y transformamos a Numpy.
# Importante: NO guardamos las clases (que estarían en data.loc[:,0])

X = np.asarray( data.loc[:,1:] )

print('\nDimensiones de los datos:', X.shape)

# Mostramos un dígito de ejemplo:

# Restaurar la forma original de la imagen
ejemplo = X[0].reshape(28, 28, 1)

plt.axis("off")
plt.title("Dígito de ejemplo")
plt.imshow(ejemplo)
plt.show()

<br>

A continuación ejecutamos el algoritmo k-means con k=10

<br>

In [ ]:
# Importamos k-means

from sklearn.cluster import KMeans


#Inicializamos el algoritmo

kmeans = KMeans(n_clusters=10, init='k-means++', n_init='auto', random_state=20)


# Realizamos el proceso de clustering sobre los datos X

clustering = kmeans.fit(X)


print("\n¡Clustering finalizado!")


<br>

Y por último comprobamos los resultados:

<br>

In [ ]:

# Número de clusters:

print("El número de clusters es:", clustering.n_clusters)


# Vamos a mostrar los números asignados a cada cluster
# Para esto primero recuperamos las etiquetas originales

Y = np.asarray( data.loc[:,0] )


# Almacenamos las etiquetas por cluster

from collections import defaultdict
results = defaultdict(list)

for i in range(len(X)):
  results[ clustering.labels_[i] ].append( Y[i] )


# Y mostramos un histograma de cada cluster con los dígitos asignados

for c in range(10):
  print("\n\nContenido del cluster {}:".format(c))
  plt.figure(figsize=(3,2))
  plt.hist(results[c], bins=10)
  plt.xticks(range(10))
  plt.show()

# Ejemplo 3 - PCA

Vamos ahora a reducir la dimensionalidad de los datos de MNIST usando el algoritmo [PCA de Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

<br>

En este caso simplemente estudiaremos cómo aplicar este algoritmo para reducir cada dígito, compuesto por 784 características, a solamente 2 características.


<br>

Empezamos por inicializar el algoritmo PCA y procesar los datos de MNIST:

In [ ]:
#Importamos la librería
from sklearn.decomposition import PCA

# Inicializamos el método PCA y establecemos que nos devuelva elementos de 2 dimensiones
PCA_Mapping = PCA(n_components=2, random_state=2)

# Ajustamos el método con los datos X de MNIST
PCA_Mapping = PCA_Mapping.fit(X)

# Mapeamos los datos:
X_clust_PCA = PCA_Mapping.transform(X)


#Mostramos los resultados:
print("Antes de PCA tenemos {} elementos con {} características cada uno".format(X.shape[0], X.shape[1]))

print("Tras PCA tenemos {} elementos con {} características cada uno".format(X_clust_PCA.shape[0], X_clust_PCA.shape[1]))

<br>

Por último representamos en una gráfica los dígitos utilizando la nueva representación con 2 dimensiones obtenida mediante PCA:

<br>

In [ ]:
# Reducimos el número elementos a mostrar para crear una gráfica más clara
num_elementos = 5000

plt.scatter(X_clust_PCA[:num_elementos, 0],
            X_clust_PCA[:num_elementos, 1],
            c=Y[:num_elementos],
            cmap=plt.cm.Set1, edgecolor="k")
plt.show()